**Load Dependencies**


In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Dense
from tensorflow.keras.layers import Flatten 
from tensorflow.keras import regularizers
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
from sklearn.model_selection import train_test_split


**Load Marsyas MFCC Dataset**


In [ ]:
file_path = "/data/mfcc_gtzan.json"

with open(file_path, "r") as f:
    data = json.load(f)

**Transforms Labels and MFCCs to Numpy Arrays**

In [ ]:
X = np.array(data['mfcc'])
y = np.array(data['labels'])


**Split Data Set Into Training, Validation and Testing Sets**


In [ ]:
TEST_SIZE = 0.25
VALIDATION_SIZE = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)
# Subdivide Training Set Into Validation and Training Set 
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=VALIDATION_SIZE)


**Construct CNN** 

In [12]:


input_shape = (X.train.shape[1], X_train.shape[2],1)
num_genres = 10

model = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 4)))
model.add(Conv2D(64, (3, 5), activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (2, 2), activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dense(num_genres, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
             optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
             metrics=['accuracy'])

model.summary()


**Fit Model Over Training Set**


In [ ]:
BATCH_SIZE = 32
EPOCHS = 50
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=BATCH_SIZE, epochs=EPOCHS) 
